In [1]:
import warnings
import numpy as np
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.compose import ColumnTransformer, make_column_transformer
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV

warnings.filterwarnings(action="ignore")
pd.set_option("float_format", '{:.2f}'.format)

In [2]:
housing = pd.read_csv("https://raw.githubusercontent.com/stivenlopezg/DS-ONLINE-64/main/data/housing.csv")
housing.head()

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value,ocean_proximity
0,-122.23,37.88,41.00,880.00,129.00,322.00,126.00,8.33,452600.00,NEAR BAY
1,-122.22,37.86,21.00,7099.00,1106.00,2401.00,1138.00,8.30,358500.00,NEAR BAY
2,-122.24,37.85,52.00,1467.00,190.00,496.00,177.00,7.26,352100.00,NEAR BAY
3,-122.25,37.85,52.00,1274.00,235.00,558.00,219.00,5.64,341300.00,NEAR BAY
4,-122.25,37.85,52.00,1627.00,280.00,565.00,259.00,3.85,342200.00,NEAR BAY


In [3]:
price = housing.pop("median_house_value")

X_train, X_test, y_train, y_test = train_test_split(housing, price,
                                                    test_size=0.3, random_state=42)

X_test, X_valid, y_test, y_valid = train_test_split(X_test, y_test,
                                                    test_size=0.5, random_state=42)

In [4]:
X_valid.to_csv("../data/housing_new.csv", index=False, header=True)
y_valid.to_csv("../data/housing_labels.csv", index=False, header=True)

## Preprocesamiento

* Preprocesamiento:
    * numéricas:
        * Tratar datos missing.
        * Tratar datos atípicos.
        * Escalar los datos.
        * Crear variables a partir de las originales (operaciones como suma, resta, multipliación, etc, y binning o discretización).
    * categóricas:
        * Tratar datos missing.
        * Codificar las variables.


In [7]:
numerical_features = housing.select_dtypes(include="number").columns.tolist()
categorical_features = [col for col in housing.columns if col not in numerical_features]

In [9]:
for col in ["latitude", "longitude"]:
    numerical_features.remove(col)

In [22]:
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import RobustScaler, OneHotEncoder

# numeric_preprocessing = Pipeline(steps=[
#     ("imputer_num", SimpleImputer(strategy="median")),
#     ("scaler", RobustScaler())])

numeric_preprocessing = make_pipeline(SimpleImputer(strategy="median"), RobustScaler())

In [18]:
# numeric_preprocessing.fit(X_train[numerical_features])

Pipeline(steps=[('simpleimputer', SimpleImputer(strategy='median')),
                ('robustscaler', RobustScaler())])

In [21]:
# numeric_preprocessing["simpleimputer"].statistics_

array([  29.    , 2131.    ,  437.    , 1168.    ,  411.    ,    3.5391])

In [23]:

categoric_preprocessing = make_pipeline(SimpleImputer(strategy="most_frequent"),
                                        OneHotEncoder(sparse=False, handle_unknown="ignore"))

In [35]:
# X_train.head(10)

In [34]:
# categoric_preprocessing.fit_transform(X_train[categorical_features])[:10]

In [41]:
# "passthrough"

# preprocessing = ColumnTransformer(transformers=[
#     ("num_preprocessing", numeric_preprocessing, numerical_features),
#     ("cat_preprocessing", categoric_preprocessing, categorical_features)
# ], remainder="drop")

preprocessing = make_column_transformer((numeric_preprocessing, numerical_features),
                                        (categoric_preprocessing, categorical_features), remainder="drop")

In [43]:
linear_reg = make_pipeline(preprocessing, LinearRegression())

gbm = make_pipeline(preprocessing, GradientBoostingRegressor())

In [37]:
## Validacion cruzada

rmse = {}

rmse["linear_reg"] = np.sqrt(-cross_val_score(estimator=linear_reg, X=X_train, y=y_train,
                                      scoring="neg_mean_squared_error", cv=5))
rmse["gradient_boosting"] = np.sqrt(-cross_val_score(estimator=gbm, X=X_train, y=y_train,
                                                     scoring="neg_mean_squared_error", cv=5))

In [38]:
rmse = pd.DataFrame.from_dict(rmse)
rmse

,linear_reg,gradient_boosting
0,71147.54,63424.07
1,68925.94,62590.47
2,68672.22,62270.06
3,69153.35,62091.05
4,72714.76,64695.90


## Optimización de hiperparametros

In [44]:
params = {
    "gradientboostingregressor__loss": ['ls', 'lad', 'huber'],
    "gradientboostingregressor__n_estimators": [50, 100, 150],
    'gradientboostingregressor__criterion': ["friedman_mse", "mse"]
}

gbm_cv = GridSearchCV(estimator=gbm, param_grid=params,
                      scoring="neg_mean_squared_error", cv=5, n_jobs=-1).fit(X_train, y_train)

In [47]:
print(f'Los mejores hiperparametros son: \n')
for key, value in gbm_cv.best_params_.items():
    print(f'{key}: {value}')
print(f'La raiz del error cuadratico medio: {np.round(np.sqrt(-gbm_cv.best_score_), 2)}')

Los mejores hiperparametros son: 

gradientboostingregressor__criterion: mse
gradientboostingregressor__loss: ls
gradientboostingregressor__n_estimators: 150
La raiz del error cuadratico medio: 62249.75


In [48]:
best_model = gbm_cv.best_estimator_

print(f"El RMSE en datos no observados es: {np.sqrt(mean_squared_error(y_test, best_model.predict(X_test)))}")

El RMSE en datos no observados es: 64305.67674989066


In [49]:
import joblib

joblib.dump(value=best_model, filename="../models/gboosting")

['../models/gboosting']

In [50]:
best_model = None

## Hacer predicciones sobre datos nuevos

In [57]:
new_data = pd.read_csv("../data/housing_new.csv")
# new_data.drop(["latitude", "longitude"], axis=1, inplace=True)

new_data.head()

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,ocean_proximity
0,-119.65,36.51,30.00,1671.00,319.00,966.00,282.00,3.13,INLAND
1,-117.01,33.97,18.00,4775.00,886.00,1868.00,836.00,2.34,INLAND
2,-118.37,34.15,29.00,2630.00,617.00,1071.00,573.00,3.37,<1H OCEAN
3,-117.11,33.12,46.00,52.00,13.00,59.00,13.00,3.88,<1H OCEAN
4,-119.06,35.36,9.00,1228.00,234.00,409.00,212.00,4.35,INLAND


In [58]:
gbm = joblib.load(filename="../models/gboosting")

In [59]:
new_data["housing_median_value_pred"] = gbm.predict(new_data)

new_data

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,ocean_proximity,housing_median_value_pred
0,-119.65,36.51,30.00,1671.00,319.00,966.00,282.00,3.13,INLAND,102340.88
1,-117.01,33.97,18.00,4775.00,886.00,1868.00,836.00,2.34,INLAND,102785.56
2,-118.37,34.15,29.00,2630.00,617.00,1071.00,573.00,3.37,<1H OCEAN,274065.40
3,-117.11,33.12,46.00,52.00,13.00,59.00,13.00,3.88,<1H OCEAN,230144.82
4,-119.06,35.36,9.00,1228.00,234.00,409.00,212.00,4.35,INLAND,168768.59
...,...,...,...,...,...,...,...,...,...,...
3091,-121.60,36.81,18.00,1575.00,230.00,751.00,219.00,5.22,<1H OCEAN,242421.78
3092,-122.43,37.75,52.00,2960.00,623.00,1191.00,589.00,3.95,NEAR BAY,361567.83
3093,-117.27,33.77,16.00,2876.00,576.00,1859.00,545.00,2.09,<1H OCEAN,123130.06
3094,-118.27,33.97,39.00,2569.00,688.00,2601.00,630.00,2.08,<1H OCEAN,148502.89
